# maps for a more complex intersection (example_1)

## this location and the data
a five way intersection, slightly skewed, with traffic islands, surrounded by a bunch of buildings and some sidewalk grass. 

![image of the intersection](pics/example_1_0.PNG)

The data in OSM has the sidewalk mapped as lines

![osm data image](pics/example_1_1.PNG)

## the parameters
use this set of parameters to make an A4 map, with some weird styles:

(the same as the param.json)
```json
{
    "size_code": "A4", 
    "center_lat": 48.58630,
    "center_lon": 7.76322,
    "line_width": 1, 
    "line_gap": 3, 
    "area_gap": 5, 
    "icon_size": 4, 
    "icon_gap": 3,
    "building_level": "detailed",
    "overlay_pref": "direct",
    "object_list": ["building", "green"],
    "style_list": ["area_8", "sidewalk_area_default"],
    "lane_width": 3.5,  // don't change this
    "epsg": "3857"  // don't change this either
}

## running the scripts

1. `00a_prepare.py` makes some basic preparations (making folders, and making the extent for the processing and the map)

In [ ]:
# change working directory to the scripts location
import os
curr = os.getcwd()
target = '/home/jovyan/notebook_scripts/'
if curr != target:
    %cd '../notebook_scripts'
os.getcwd()

In [ ]:
%run 00a_prepare.py example_1

2. `00b_pre_process.py` extracts the data from the `map.osm` and does some basic proeesings on the street, including estimating the lane count of the street

    data extracted from osm:
    - street (line)
    - pedestrian crossing (point)
    - sidewalk (area or line, depending on availability)
    - buildings (area)
    - green areas (area)
    - bus stop (point)

In [ ]:
%run 00b_pre_process.py example_1

after this, normally you check the data to see if the lane counts are currectly labelled and correct it if you need to.

3. `01_street.py` creates the street boundary line (the curb line) from the street and/or sidewalk data.

if there is sidewalk area data available, the street boundary would use the sidewalk boundary because that is more accurate. Although sidewalk as an area can only be properly displayed on A3 (1:500) maps because minimal area dimension and gap requirements etc.

if there is no sidewalk area data available like this location, the lane count estimated in the previous step is used to create an area feature of the street, the smoothed boundary of this area feature becomes the curb line.

In [ ]:
warnings.filterwarnings('ignore')
%run 01_street.py example_1

4. `02_pedestrian_crossing.py` make pedestrian crossing lines from the points. 

In [ ]:
%run 02_pedestrian_crossing.py example_1

quick plot of the curb line, the islands and the pedestrian crossings

In [ ]:
import geopandas as gpd

streets = gpd.read_file("../example_1/data/1000/street_boundary_filled.geojson")
zebras = gpd.read_file("../example_1/data/1000/crossing_lines.geojson")
islands = gpd.read_file("../example_1/data/1000/islands_full.geojson")
extent = gpd.read_file("../example_1/data/extent_A4.geojson")

ax = streets.plot()
zebras.plot(ax=ax, color='r')
islands.plot(ax=ax, color='lightgray')
minx, miny, maxx, maxy = extent.total_bounds
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

this location no traffic island, and these islands are configured differently. 

there is one kind of configuration (a "cut through") where there the traffic island is cut into two parts with raised curbs along the middle section where the pedestrian pass. like this:

![image of the cut through](pics/example_1_2.PNG)

and there is another kind of configuration (a "slope") where there are no additional curbs:

![image of the slope island](pics/example_1_3.PNG)


there is a choice, to differenciate these two kind of configuration and present them differently on the map. This step is not mandatory, if such information is not important to your mobility needs (e.g. when your dog will leads you through the island.)

BUT THIS INVOLVES A MANUAL STEP to identify which island is a cut-through (there is NO way to know this from the data.)

if you want to do this step:

1. go to find the file `island.geojson` and visualize it 
2. for the island with a cut through, manually edit the attribute `split` into `1`
3. run the split island script

In [ ]:
%run 03_island_split.py example_1

then do the buildings and the grass areas 

In [ ]:
%run 06a_buildings.py example_1
%run 06b_green.py example_1

and a little plot of the original building location / shape and the generalized location / shape: in the "detailed" generalization scenario, it's not generalized as much.

In [ ]:
import geopandas as gpd

import warnings
warnings.filterwarnings('ignore')

building_osm = gpd.read_file('../example_1/data/buildings.geojson')
building_gen = gpd.read_file('../example_1/data/500/buildings_gen.geojson')
extent = gpd.read_file("../example_1/data/extent_A4.geojson")

ax = building_osm.plot(color='lightblue', alpha=0.5)
building_gen.plot(ax=ax, color='lightgray', alpha=0.7)
minx, miny, maxx, maxy = extent.total_bounds
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

now produce the maps.

the styles and templates that are currently included.

![default styles](pics/style_samples.PNG)

template A5:
![a5 template](pics/templatea5.PNG)

template A4 and A3:
![a3 a4](pics/templatesa4.PNG)

first install the braille font. "Braille TBFr2007 IHS HEA thermo" is used here. it can be found at [here](http://natbraille.free.fr/ressources/fonts/BrailleTBFr2007INSHEAb_thermo.ttf) and in the `templates` folder.

then run the map assembly script `07_assembly.py` that will piece together the geometries and the styles into the template.

(it will say "ERROR 6: The PNG driver does not support update access to existing datasets." but it's ok)

In [ ]:
# install fonts on this virtual machine
%mkdir ~/.local/share/fonts
%cp /home/jovyan/templates/BrailleTBFr2007INSHEAb_thermo.ttf  ~/.local/share/fonts/
!fc-cache -f -v

In [ ]:
# run map assembly script
warnings.filterwarnings('ignore')
%run 07_assembly.py example_1

## and the map!

directly open the file from the left folder tree (`example_1/image/A4.pdf`) and check it in another tab.

or run the code in the next block to have it shown here.

In [ ]:
from IPython.display import Image
Image(filename='../example_0/image/A4.png')